# Laboratorio 1 - Analisis de ventas Big Data - Ivan Gonzalo Tapia



### Consignas:
Preguntas a responder en el análisis
—————————————————————————————————————————
1. Comportamiento de las ventas en los distintos meses:

  ● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

  ● ¿Hubo algún mes que se destacó en términos de ventas?

  ● ¿Cuál es el ingreso total generado por mes?
2. Optimización de la publicidad y patrón de ventas por hora:

  ● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
  probabilidad de compra?

  ● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

  ● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
  en los distintos meses?

3. Distribución de ventas por ubicación:

  ● ¿En qué ciudades se han registrado las mayores ventas?

  ● ¿Cómo se comparan las ventas en diferentes estados o regiones?

  ● ¿Existe variación en las ventas por estado a lo largo de los meses?

4. Análisis del producto más vendido:

  ● ¿Cuál es el producto más vendido en general y en cada mes?

  ● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

5. Tendencia ventas:

  ● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
  en los diferentes meses?

  ● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

  ● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
  los fines de semana?

6. Impacto de eventos especiales en las ventas:

  ● ¿Se ha observado algún aumento o disminución significativa en las ventas en días
  cercanos a eventos especiales, como días festivos?

  ● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
  las ventas y cómo se manifestó ese impacto?

Pregunta Adicional
—————————————————————————————————————————

   - Además de las preguntas mencionadas anteriormente, te invito a pensar en una pregunta
  adicional que podría ser relevante para analizar los datos de ventas mensuales. Esta
  pregunta debe estar basada en la información que se proporciona en los conjuntos de
  datos. Puede estar relacionada con tendencias, comparaciones, patrones o cualquier otro
  aspecto que despierte tu interés. Trata de ser creativo/a y pensar en cómo podrías explorar
  aún más estos datos!


Pasos del Análisis
—————————————————————————————————————————
1. Carga de datos:
  - Cargar los datos de ventas de cada archivo CSV.
2. Limpieza de los datos:
  - Eliminar datos no numéricos.
  - Eliminar filas incompletas
3. Preparación de los datos:
  - Ajustar los tipos de datos de cada atributo
  - Extraer características importantes para el análisis como meses, horas, ciudades etc.
4. Análisis de datos:
  Ejemplo:
  - Examinar las horas del día en que las ventas son más frecuentes.
  - Determinar el mejor momento para mostrar publicidad.
5. Visualización de datos:
  - Utilizar gráficos y visualizaciones para representar los resultados de los análisis.
  - Crear gráficos de barras, líneas, mapas, tablas, etc, según corresponda.

## Import de librerias y recursos a utilizar

In [1]:
# Librerias necesarias para el desarrollo del proyecto
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
# Directorio propio para importar los datasets necesarios

path = '/content/drive/MyDrive/UPSO ML & BD/Dataset de ventas2'

archivos = os.listdir(path)
archivos

['Dataset_de_ventas_Abril.csv',
 'Dataset_de_ventas_Agosto.csv',
 'Dataset_de_ventas_Diciembre.csv',
 'Dataset_de_ventas_Enero.csv',
 'Dataset_de_ventas_Febrero.csv',
 'Dataset_de_ventas_Julio.csv',
 'Dataset_de_ventas_Junio.csv',
 'Dataset_de_ventas_Marzo.csv',
 'Dataset_de_ventas_Mayo.csv',
 'Dataset_de_ventas_Noviembre.csv',
 'Dataset_de_ventas_Octubre.csv',
 'Dataset_de_ventas_Septiembre.csv']

## Preparacion y limpieza de lo datos

In [3]:
# Ahora voy a crear un diccionario para guardar estos archivos(datasets), para luego hacer un df

dataframes = {}

for archivo in archivos:
  if archivo.endswith('csv'):
    mes = archivo.split('_')[3].split('.')[0] # obtener el nombre del mes solo
    #aca obtenemos la ruta completa de donde se encuentra el csv de cada mes
    path_completo = os.path.join(path, archivo) # obtener ruta de acceso completa
    df = pd.read_csv(path_completo)   #cargar el archivo csv en un df
    #vamos a trabajar sobre una copia del df original
    df_ventas = df.copy()


    # SANITIZAR CADA DF ANTES DE CARGARLO
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors = 'coerce') #errors = coerce transfora lo
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors = 'coerce')
    #esta es una mascara que me genera un true si hay un nan, y sino hay nan me pone un false
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]
    #aca elimino los nan's
    df_ventas = df_ventas.dropna()
    #aca voy a pasar todos los datos al tipo q corresponda y agregar las columnas que sean necesarias
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int) #esta funcion me transforma todos los valores a numericos a int
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float) #esta funcion me transforma todos los valores a numericos a float

    # FINALIZA SANITIZACION

    # CREO COLUMNAS NECESARIAS PARA ANALISIS
    df_ventas['Fecha de Pedido'] = pd.to_datetime(df_ventas['Fecha de Pedido'], format='%m/%d/%y %H:%M')

    # Obtener la hora y minutos como número entero de 4 dígitos (por ejemplo, 1430)
    df_ventas['Hora de Pedido'] = df_ventas['Fecha de Pedido'].dt.strftime('%H%M').astype(int)
    df_ventas['Estado'] = df_ventas['Dirección de Envio'].str.split(', ').str[-1].str.split(' ').str[0]
    df_ventas['Ciudad'] = df_ventas['Dirección de Envio'].str.split(', ').str[1]
    df_ventas['Dia'] = df_ventas['Fecha de Pedido'].dt.strftime('%d').astype(int)
    df_ventas['Mes'] = df_ventas['Fecha de Pedido'].dt.strftime('%m').astype(int)
    #FINALIZA CREACION DE COLUMNAS

    #aca sucede la magia, vamos agregando cada mes al df ventas
    #a cada mes le cargo su dataset correspondiente, por eso uso la variable mes
    dataframes[mes] = df_ventas


In [5]:
dataframe_e = dataframes['Enero']
dataframe_e.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia,Mes
0,141234,iPhone,1,700.00,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",2125,MA,Boston,22,1
1,141235,Lightning Charging Cable,1,14.95,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",1415,OR,Portland,28,1
2,141236,Wired Headphones,2,11.99,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",1333,CA,San Francisco,17,1
3,141237,27in FHD Monitor,1,149.99,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",2033,CA,Los Angeles,5,1
4,141238,Wired Headphones,1,11.99,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",1159,TX,Austin,25,1


In [6]:
dataframe_e.tail()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia,Mes
9718,150497,20in Monitor,1,109.99,2019-01-26 19:09:00,"95 8th St, Dallas, TX 75001",1909,TX,Dallas,26,1
9719,150498,27in FHD Monitor,1,149.99,2019-01-10 22:58:00,"403 7th St, San Francisco, CA 94016",2258,CA,San Francisco,10,1
9720,150499,ThinkPad Laptop,1,999.99,2019-01-21 14:31:00,"214 Main St, Portland, OR 97035",1431,OR,Portland,21,1
9721,150500,AAA Batteries (4-pack),2,2.99,2019-01-15 14:21:00,"810 2nd St, Los Angeles, CA 90001",1421,CA,Los Angeles,15,1
9722,150501,Google Phone,1,600.00,2019-01-13 16:43:00,"428 Cedar St, Boston, MA 02215",1643,MA,Boston,13,1


In [7]:
dataframes['Febrero']

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia,Mes
0,150502,iPhone,1,700.00,2019-02-18 01:35:00,"866 Spruce St, Portland, ME 04101",135,ME,Portland,18,2
1,150503,AA Batteries (4-pack),1,3.84,2019-02-13 07:24:00,"18 13th St, San Francisco, CA 94016",724,CA,San Francisco,13,2
2,150504,27in 4K Gaming Monitor,1,389.99,2019-02-18 09:46:00,"52 6th St, New York City, NY 10001",946,NY,New York City,18,2
3,150505,Lightning Charging Cable,1,14.95,2019-02-02 16:47:00,"129 Cherry St, Atlanta, GA 30301",1647,GA,Atlanta,2,2
4,150506,AA Batteries (4-pack),2,3.84,2019-02-28 20:32:00,"548 Lincoln St, Seattle, WA 98101",2032,WA,Seattle,28,2
...,...,...,...,...,...,...,...,...,...,...,...
12031,162004,Apple Airpods Headphones,1,150.00,2019-02-12 22:02:00,"227 Church St, San Francisco, CA 94016",2202,CA,San Francisco,12,2
12032,162005,AAA Batteries (4-pack),2,2.99,2019-02-04 20:44:00,"417 Jefferson St, Los Angeles, CA 90001",2044,CA,Los Angeles,4,2
12033,162006,USB-C Charging Cable,1,11.95,2019-02-24 06:31:00,"498 8th St, Atlanta, GA 30301",631,GA,Atlanta,24,2
12034,162007,USB-C Charging Cable,1,11.95,2019-02-24 19:09:00,"715 7th St, Dallas, TX 75001",1909,TX,Dallas,24,2


In [8]:
dataframes['Diciembre']

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia,Mes
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",1,NY,New York City,30,12
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",703,NY,New York City,29,12
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",1821,NY,New York City,12,12
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",1513,CA,San Francisco,22,12
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",1238,GA,Atlanta,18,12
...,...,...,...,...,...,...,...,...,...,...,...
25112,319666,Lightning Charging Cable,1,14.95,2019-12-11 20:58:00,"14 Madison St, San Francisco, CA 94016",2058,CA,San Francisco,11,12
25113,319667,AA Batteries (4-pack),2,3.84,2019-12-01 12:01:00,"549 Willow St, Los Angeles, CA 90001",1201,CA,Los Angeles,1,12
25114,319668,Vareebadd Phone,1,400.00,2019-12-09 06:43:00,"273 Wilson St, Seattle, WA 98101",643,WA,Seattle,9,12
25115,319669,Wired Headphones,1,11.99,2019-12-03 10:39:00,"778 River St, Dallas, TX 75001",1039,TX,Dallas,3,12


## Analisis y graficos de datos

In [ ]:
# Funcion para obtener los datos de dia y hora de los pedidos

def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = df_ventas['Fecha de Pedido'].dt.hour
  datos['Dia'] = df_ventas['Fecha de Pedido'].dt.day
  return datos

In [32]:

# Ordenar los meses
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

# Crear una lista para almacenar los totales de ventas por mes
totales_ventas = []

# Itera a través de los meses
for mes in meses:
    # Obtén el DataFrame del mes actual
    dataframe_mes = dataframes[mes]

    # Aplica la función para extraer datos y calcular el total de ventas del mes
    dataframe_mes = extraer_datos(dataframe_mes)
    total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()

    # Agrega el total de ventas a la lista
    totales_ventas.append(total_ventas_mes)

# Ahora, totales_ventas contendrá los totales de ventas en el mismo orden que la lista de meses
print("Meses:", meses)
print("Totales de Ventas:", totales_ventas)


NameError: ignored

### 1. Comportamiento de las ventas en los distintos meses:

  ● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

  ● ¿Hubo algún mes que se destacó en términos de ventas?

  ● ¿Cuál es el ingreso total generado por mes?

In [ ]:
# Creo la figura con plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x=meses,
    y=totales_ventas,
    yaxis="y",
    name="Unidades vendidas",
    marker =dict(color=totales_ventas,colorscale= 'RdYlGn', colorbar= dict(title= "Total de ventas")),
    text = totales_ventas,
    textposition = 'auto'

))

fig.update_layout(
    title="Unidades vendidas por mes",
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side = "left")
)

fig.show()

In [ ]:
import plotly.graph_objects as go

sizes = []
colors = []

#Hago un for para recorrer la cantidad total de ventas, y voy a dividir por una constante(1000), para hacer las burbujas de color acorde a la cantidad de unidades vendidas por mes
for total in totales_ventas:
  s = 35
  c = total / 1000
  sizes.append(s)
  colors.append(c)

fig = go.Figure(data=[go.Scatter(
    x=meses,
    y=totales_ventas,
    mode='markers',
    marker=dict(
        color=colors,
        size= sizes,
        showscale=True,
        colorbar= dict(title= "Ventas en miles")
        )
)])

fig.update_layout(

  title="Cantidad de productos vendidos por mes",
  xaxis=dict(title="Meses", tickvals=meses),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
  yaxis=dict(title="Unidades vendidas", side="left"),
  legend=dict(y=0.3, yanchor='bottom')
)


fig.show()

In [ ]:
ingresos_totales = []

for mes in meses:
  total_ventas_mes = (dataframes[mes]['Cantidad Pedida']*dataframes[mes]['Precio Unitario']).sum().round(2)
  ingresos_totales.append(total_ventas_mes)

# Crear el objeto figura
fig = go.Figure()

#Configuro titulos y ejes del grafico
fig.add_trace(go.Bar(
        x = meses,
        y = ingresos_totales,
        yaxis= 'y',
        name = 'Ingreso Total',
        marker = dict(color= ingresos_totales, colorscale='balance', colorbar=dict(title = 'Total de ventas')),
        text = ingresos_totales,
        textposition = 'outside',
)
)

fig.update_layout(
    title = 'Ingresos totales por mes',
    xaxis = dict(title='Meses'),
    yaxis = dict(title='Ingreso total',side='left')
)

#Muestro el grafico
fig.show()

### 2. Optimización de la publicidad y patrón de ventas por hora:

  ● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
  probabilidad de compra?

  ● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

  ● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
  en los distintos meses?

In [ ]:
# Creo el objeto figura
Horas = list(range(24))
fig = go.Figure()


for mes in meses:

  Ventas_por_hora= dataframes[mes].groupby('Hora')['Cantidad Pedida'].sum()

  # Agrego al objeto figura un grafico de de linea con las horas y las ventas por hora
  fig.add_trace(go.Scatter(

      x=Horas,
      y=Ventas_por_hora,
      mode='lines',
      name=mes,
      line=dict(shape='spline', smoothing=1.3, width=2)))


  # Configuro los ejes y los títulos
fig.update_layout(

    title="Unidades vendidas por hora",
    xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom')),



fig.show()

In [ ]:
fig = go.Figure()

horas_analizadas = [12,19,14]

for hora in horas_analizadas:
  total_productos_vendidos = []

  for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframes_mes = extraer_datos(dataframe_mes)

    ventas_hora = dataframe_mes[dataframe_mes['Hora'] == hora]

    total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].sum())

  fig.add_trace(go.Scatter(

      x=meses,
      y=total_productos_vendidos,
      mode='lines',
      name=f'{hora} horas',
      line = dict(width= 2, shape= 'spline')
  ))


# Configuro los ejes y los títulos
fig.update_layout(

  title="Unidades vendidas por hora",
  xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
  yaxis=dict(title="Unidades vendidas", side="left"),
  legend=dict(y=0.3, yanchor='bottom')
)


fig.show()

In [ ]:


fig = go.Figure()
fig.add_trace(go.Bar(
    x=meses,
    y=total_productos_vendidos,
    name='Total de unidades vendidas',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=meses,
    y=Ventas_por_hora,
    name='Ventas por hora',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

### 2 - Optimizacion de las ventas por hora


In [ ]:
ventas_hora = []
# Agrupo las ventas por hora y calculo el total de ventas por hora
for mes in meses:
  ventas_por_hora = dataframes[mes].groupby(dataframes[mes]['Hora de Pedido'])['Cantidad Pedida'].sum().reset_index()
  ventas_hora.append(ventas_por_hora)
# Encuentro la hora con mas ventas
hora_max_ventas = ventas_por_hora.loc[ventas_por_hora['Cantidad Pedida'].idxmax()]['Hora de Pedido']
print(f"La hora ideal para presentar la publicidad es a las {hora_max_ventas} horas.")

La hora ideal para presentar la publicidad es a las 1145 horas.


In [ ]:
# Crear un gráfico de barras apiladas para visualizar los patrones de ventas por hora a lo largo del año
import plotly.express as px
ventas_ciudad = []
# Agrupar las ventas por ciudad y calcular el total de ventas por ciudad
for mes in meses:
  ventas_por_ciudad = dataframes[mes].groupby('Ciudad')['Cantidad Pedida'].sum().reset_index()
  ventas_ciudad.append(ventas_por_ciudad)

# Crear un gráfico de barras para visualizar las ventas por ciudad
fig = px.bar(ventas_por_ciudad, x='Ciudad', y='Cantidad Pedida', labels={'Cantidad Pedida': 'Ventas'})
fig.update_layout(title="Ventas por ciudad")
fig.show()

### 3. Distribución de ventas por ubicación:

  ● ¿En qué ciudades se han registrado las mayores ventas?

  ● ¿Cómo se comparan las ventas en diferentes estados o regiones?

  ● ¿Existe variación en las ventas por estado a lo largo de los meses?


In [ ]:
# Voy a investigar las ventas por ciudad, obteniendo los datos de la columna ciudad que cree en el procesamiento de datos
for mes in meses:
  ventas_por_ciudad =  dataframes[mes].groupby('Ciudad')['Cantidad Pedida'].sum().reset_index(inplace=False)

ventas_por_ciudad

,Ciudad,Cantidad Pedida
0,Atlanta,2224
1,Austin,1520
2,Boston,3029
3,Dallas,2300
4,Los Angeles,4373
5,New York City,3828
6,Portland,1859
7,San Francisco,6786
8,Seattle,2202


In [ ]:
import plotly.graph_objs as go

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']

# Obtener una paleta de colores similar a la usada en Plotly Express
colores_ciudad = px.colors.qualitative.Set1

# Crear un diccionario que mapee cada ciudad a su color correspondiente
color_por_ciudad = {}
for i, ciudad in enumerate(ciudades):
    color = colores_ciudad[i % len(colores_ciudad)]  # Ciclar colores de la paleta
    color_por_ciudad[ciudad] = color

# Agregar el gráfico de barras con colores por ciudad
for ciudad in ciudades:
    color = color_por_ciudad[ciudad]  # Obtener el color correspondiente a la ciudad
    fig.add_trace(go.Bar(
        x=[ciudad],
        y=[cantidades[ciudades == ciudad].values[0]],  # Obtener la cantidad para la ciudad específica
        name=ciudad,
        marker_color=color  # Asignar el color al gráfico de barras
    ))

# Agregar el gráfico de dispersión con los máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color = "red"

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()


In [ ]:
import plotly.graph_objs as go

# Calcular los máximos por ciudad
maximos_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].max().reset_index()

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']

# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=ciudades,
    y=cantidades,
    mode='lines',
    name="Variacion de unidad vendidas",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color_max = color_por_ciudad[ciudad_max]  # Obtener el color correspondiente a la ciudad máxima
    fig.add_trace(go.Scatter(
        x=[ciudad_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({ciudad_max})',
        marker=dict(size=10, color=color_max)  # Asignar el color al gráfico de dispersión
    ))

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)
fig.show()






In [ ]:
import plotly.express as px

# Calcular las proporciones de ventas por ciudad
ventas_por_ciudad = ventas_por_ciudad.sort_values(by='Cantidad Pedida', ascending=False)
ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']
total_ventas = cantidades.sum()
proporciones = cantidades / total_ventas

# Crear un DataFrame para el gráfico
df_pie = pd.DataFrame({'Ciudad': ciudades, 'Ventas': cantidades, 'Proporción': proporciones})

# Crear el gráfico de pastel interactivo
fig = px.pie(df_pie, names='Ciudad', values='Ventas', color='Ciudad', color_discrete_map=color_por_ciudad,
             labels={'Ciudad': 'Ciudad', 'Ventas': 'Ventas'},
             hover_data=['Proporción'],
             title='Distribución de Ventas por Ciudad')

# Agregar etiquetas de porcentaje
fig.update_traces(textinfo='percent+label')

# Destacar la ciudad con más ventas
fig.update_traces(pull=[0.1] + [0] * (len(ciudades) - 1))

# Configurar el diseño
fig.update_layout(legend_title_text='Ciudades')

# Mostrar el gráfico
fig.show()




In [ ]:
import plotly.graph_objs as go

# Calcular los máximos por ciudad
maximos_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].max().reset_index()

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']
colores_ciudad = px.colors.qualitative.Set1
# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=ciudades,
    y=cantidades,
    mode='lines',
    name="Variacion de unidad vendidas",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color_max = color_por_ciudad[ciudad_max]  # Obtener el color correspondiente a la ciudad máxima
    fig.add_trace(go.Scatter(
        x=[ciudad_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({ciudad_max})',
        marker=dict(size=10, color=color_max)  # Asignar el color al gráfico de dispersión
    ))


# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)
fig.show()
# Crear el gráfico de pastel interactivo
fig_pie = px.pie(df_pie, names='Ciudad', values='Ventas', color='Ciudad', color_discrete_map=color_por_ciudad,
             labels={'Ciudad': 'Ciudad', 'Ventas': 'Ventas'},
             hover_data=['Proporción'],
             title='Distribución de Ventas por Ciudad')

# Agregar etiquetas de porcentaje
fig_pie.update_traces(textinfo='percent+label')

# Destacar la ciudad con más ventas
fig_pie.update_traces(pull=[0.1] + [0] * (len(ciudades) - 1))

# Configurar el diseño
fig_pie.update_layout(legend_title_text='Ciudades')

# Mostrar el gráfico

fig_pie.show()

fig_bar = go.Figure()
for ciudad in ciudades:
    color = color_por_ciudad[ciudad]  # Obtener el color correspondiente a la ciudad
    fig_bar.add_trace(go.Bar(
        x=[ciudad],
        y=[cantidades[ciudades == ciudad].values[0]],  # Obtener la cantidad para la ciudad específica
        name=ciudad,
        marker_color=color  # Asignar el color al gráfico de barras
    ))

# Configurar ejes y título
fig_bar.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig_bar.show()

In [ ]:
# Ahora voy a investigar las ventas totales por estado
for mes in meses:
  ventas_por_estado = dataframes[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado = ventas_por_estado.reset_index(inplace=False)

ventas_por_estado

,Estado,Cantidad Pedida
0,CA,11159
1,GA,2224
2,MA,3029
3,ME,352
4,NY,3828
5,OR,1507
6,TX,3820
7,WA,2202


In [ ]:
import plotly.graph_objs as go

# Calcular los máximos por estado
maximos_por_estado = ventas_por_estado.groupby('Estado')['Cantidad Pedida'].max().reset_index()
maximos_por_estado = maximos_por_estado.sort_values(by='Cantidad Pedida', ascending=True)
# Crear el gráfico
fig = go.Figure()

estados = ventas_por_estado['Estado']
cantidad = ventas_por_estado['Cantidad Pedida']

# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=estados,
    y=cantidad,
    mode='lines',
    name="Ciudad",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por estado
for _, row in maximos_por_estado.iterrows():
    estado_max = row['Estado']
    cantidad_max = row['Cantidad Pedida']
    fig.add_trace(go.Scatter(
        x=[estado_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({estado_max})',
        marker=dict(size=10)
    ))

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por estado",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()

In [ ]:
import plotly.express as px


# Hago un plot de Estados Unidos
fig = px.choropleth(ventas_por_estado, locations='Estado', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="RdYlGn",
                           scope="usa",
                          title= 'Ventas por estado'
                          )

fig.show()

In [ ]:
ventas_por_estado_mes = {}
estados = []


fig = go.Figure()

for estado in dataframes['Enero']['Estado']:
  if estado not in estados:
    estados.append(estado)

for mes in meses:
  ventas_por_estado_mes[mes] = dataframes[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado_mes[mes] = ventas_por_estado_mes[mes].reset_index(inplace=False)

# Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  fig.add_trace(go.Bar(
      x=ventas_por_estado_mes[mes]['Estado'],
      y=ventas_por_estado_mes[mes]['Cantidad Pedida'],
      name= mes,
  ));

# Configuro ejes y los títulos
fig.update_layout(
    title="Total de productos vendidos por mes en cada estado",
    xaxis=dict(title="Estados"),
    yaxis=dict(title="Ventas", side="left"),
);

fig.show()

In [ ]:
import pandas as pd

# Supongamos que tienes un diccionario llamado "dataframes" con los DataFrames correspondientes a cada mes
# Puedes cargar los datos desde tu fuente, por ejemplo:
# dataframes = {'Enero': df_enero, 'Febrero': df_febrero, ...}

# Define una función para extraer el estado a partir de la columna "Dirección de Envío"
def obtener_estado(direccion):
    partes = direccion.split(", ")
    if len(partes) > 1:
        return partes[-1]
    else:
        return None

# Inicializa un DataFrame vacío para almacenar los resultados
df_estado_total = pd.DataFrame(columns=['Estado', 'Cantidad Pedida Total'])

# Itera a través de los DataFrames de cada mes en el diccionario
for mes, df_mes in dataframes.items():
    # Aplica la función para obtener el estado y crea una nueva columna "Estado"
    df_mes['Estado'] = df_mes['Dirección de Envio'].apply(obtener_estado)

    # Agrupa por "Estado" y suma la "Cantidad Pedida" para obtener la cantidad total por estado en este mes
    df_mes_estado_total = df_mes.groupby('Estado')['Cantidad Pedida'].sum().reset_index()

    # Concatena los resultados con el DataFrame total
    df_estado_total = pd.concat([df_estado_total, df_mes_estado_total], ignore_index=True)

# Agrupa el DataFrame total por "Estado" y suma la "Cantidad Pedida" para obtener la cantidad total por estado en todos los meses
df_estado_total = df_estado_total.groupby('Estado')['Cantidad Pedida'].sum().reset_index()

# df_estado_total ahora contiene la cantidad pedida total por cada estado para todos los meses
df_estado_total.head()






,Estado,Cantidad Pedida
0,CA 90001,33289.0
1,CA 94016,50239.0
2,GA 30301,16602.0
3,MA 02215,22528.0
4,ME 04101,2750.0


### 4. Análisis del producto más vendido:
  ● ¿Cuál es el producto más vendido en general y en cada mes?

  ● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

In [ ]:
# Creo un DataFrame general combinando con todos los meses
df_total = pd.concat(dataframes.values(), ignore_index=True)

# Calculo las ventas totales por producto
producto_mas_vendido_general = df_total.groupby('Producto')['Cantidad Pedida'].sum().idxmax()
total_ventas_producto_mas_vendido = df_total.groupby('Producto')['Cantidad Pedida'].sum().max()

print("Producto más vendido en general:", producto_mas_vendido_general)
print("Total de ventas del producto más vendido:", total_ventas_producto_mas_vendido)


Producto más vendido en general: AAA Batteries (4-pack)
Total de ventas del producto más vendido: 31017


In [ ]:
# Calcular las ventas totales por producto en el DataFrame combinado
ventas_totales = df_total.groupby('Producto')['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de barras para mostrar las ventas totales por producto
fig = go.Figure()

fig.add_trace(go.Bar(
    x=ventas_totales['Producto'],
    y=ventas_totales['Cantidad Pedida'],
    yaxis="y",
    name="Unidades vendidas",
    marker=dict(color=ventas_totales['Cantidad Pedida'], colorscale='RdYlGn', colorbar=dict(title="Total de ventas")),
    text=ventas_totales['Cantidad Pedida'],
    textposition='auto'
))

fig.update_layout(
    title="Ventas totales de todos los productos en general",
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Unidades vendidas", side="left")
)

fig.show()

In [ ]:
# Crear una lista de DataFrames mensuales
dataframes_mensuales = [dataframes[mes] for mes in meses]

# Agregar una columna 'Mes' a cada DataFrame
for i, mes in enumerate(meses):
    dataframes_mensuales[i]['Mes'] = mes

# Concatenar los DataFrames mensuales en un único DataFrame total
df_total = pd.concat(dataframes_mensuales, ignore_index=True)



In [ ]:
# Convertir la columna 'Mes' en categórica con el orden deseado
df_total['Mes'] = pd.Categorical(df_total['Mes'], categories=meses, ordered=True)

# Ordenar el DataFrame por la columna 'Mes' categórica
df_total = df_total.sort_values(by='Mes')

In [ ]:

# Calcular las ventas totales por producto y mes
ventas_totales_por_producto_mes = df_total.groupby(['Producto', 'Mes'])['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de barras apiladas para mostrar las ventas totales por producto y mes
fig_ventas_por_mes = go.Figure()

for producto in ventas_totales_por_producto_mes['Producto'].unique():
    datos_producto = ventas_totales_por_producto_mes[ventas_totales_por_producto_mes['Producto'] == producto]
    fig_ventas_por_mes.add_trace(go.Bar(
        x=datos_producto['Mes'],
        y=datos_producto['Cantidad Pedida'],
        name=producto
    ))

fig_ventas_por_mes.update_layout(
    title='Ventas totales de todos los productos por mes',
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Cantidad Pedida')
)

fig_ventas_por_mes.show()

In [ ]:
import plotly.express as px

# Calcular las ventas totales por producto y mes
ventas_totales_por_producto_mes = df_total.groupby(['Producto', 'Mes'])['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de líneas para mostrar las ventas totales por producto y mes
fig_ventas_por_mes = px.line(ventas_totales_por_producto_mes, x='Mes', y='Cantidad Pedida', color='Producto',
                             title='Ventas totales de todos los productos por mes')

fig_ventas_por_mes.update_xaxes(type='category')  # Establecer el eje X como categórico (meses)
fig_ventas_por_mes.show()

In [ ]:
import plotly.express as px

# Calcular las ventas totales por producto y mes
ventas_totales_por_producto_mes = df_total.groupby(['Producto', 'Mes'])['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de áreas para mostrar las ventas totales por producto y mes
fig_ventas_por_mes = px.area(ventas_totales_por_producto_mes, x='Mes', y='Cantidad Pedida', color='Producto',
                             title='Ventas totales de todos los productos por mes')

fig_ventas_por_mes.update_xaxes(type='category')  # Establecer el eje X como categórico (meses)
fig_ventas_por_mes.show()

In [ ]:
import plotly.graph_objs as go

# Calcular las ventas totales por producto y mes
ventas_totales_por_producto_mes = df_total.groupby(['Producto', 'Mes'])['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de dispersión para mostrar las ventas totales por producto y mes
fig_ventas_por_mes = go.Figure()

for producto in ventas_totales_por_producto_mes['Producto'].unique():
    datos_producto = ventas_totales_por_producto_mes[ventas_totales_por_producto_mes['Producto'] == producto]
    fig_ventas_por_mes.add_trace(go.Scatter(
        x=datos_producto['Mes'],
        y=datos_producto['Cantidad Pedida'],
        mode='lines+markers',  # Mostrar líneas y marcadores en el gráfico
        name=producto
    ))

fig_ventas_por_mes.update_layout(
    title='Ventas totales de todos los productos por mes',
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Cantidad Pedida')
)

fig_ventas_por_mes.show()

In [ ]:
# Calcular las ventas totales por mes
ventas_totales_por_mes = df_total.groupby(df_total['Fecha de Pedido'].dt.strftime('%B'))['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de barras para mostrar las ventas totales por mes
fig_ventas_por_mes = px.bar(ventas_totales_por_mes, x='Fecha de Pedido', y='Cantidad Pedida',
                            title='Ventas totales de todos los productos por mes',
                            labels={'Fecha de Pedido': 'Mes', 'Cantidad Pedida': 'Cantidad Vendida'})

# Configurar el rango del eje Y para mostrar solo hasta 5000
fig_ventas_por_mes.update_yaxes(range=[0, 50000])

fig_ventas_por_mes.show()

In [ ]:
productos_mas_vendidos_por_mes = []

for mes in meses:
    # Obtén el DataFrame del mes actual
    dataframe_mes = dataframes[mes]

    # Agrupa por producto y calcula las ventas totales por producto
    ventas_por_producto = dataframe_mes.groupby('Producto')['Cantidad Pedida'].sum()

    # Encuentra el producto más vendido del mes
    producto_mas_vendido_mes = ventas_por_producto.idxmax()
    total_ventas_producto_mas_vendido_mes = ventas_por_producto.max()

    productos_mas_vendidos_por_mes.append((mes, producto_mas_vendido_mes, total_ventas_producto_mas_vendido_mes))

# Imprime la lista de productos más vendidos por mes
for mes, producto, total_ventas in productos_mas_vendidos_por_mes:
    print(f"Mes: {mes}, Producto más vendido: {producto}, Total de ventas: {total_ventas}")

Mes: Enero, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 1596
Mes: Febrero, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 1975
Mes: Marzo, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2478
Mes: Abril, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2936
Mes: Mayo, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2931
Mes: Junio, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2160
Mes: Julio, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2366
Mes: Agosto, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2019
Mes: Septiembre, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 1927
Mes: Octubre, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 3362
Mes: Noviembre, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 3027
Mes: Diciembre, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 4240


### 5. Tendencia ventas:

  ● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
  en los diferentes meses?

  ● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

  ● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
  los fines de semana?

In [9]:
df_5 = pd.concat(dataframes.values(), ignore_index=True)
df_5.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia,Mes
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",846,TX,Dallas,19,4
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",2230,MA,Boston,7,4
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",1438,CA,Los Angeles,12,4
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",1438,CA,Los Angeles,12,4
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",927,CA,Los Angeles,30,4


In [10]:
df_5.tail()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia,Mes
185945,259353,AAA Batteries (4-pack),3,2.99,2019-09-17 20:56:00,"840 Highland St, Los Angeles, CA 90001",2056,CA,Los Angeles,17,9
185946,259354,iPhone,1,700.00,2019-09-01 16:00:00,"216 Dogwood St, San Francisco, CA 94016",1600,CA,San Francisco,1,9
185947,259355,iPhone,1,700.00,2019-09-23 07:39:00,"220 12th St, San Francisco, CA 94016",739,CA,San Francisco,23,9
185948,259356,34in Ultrawide Monitor,1,379.99,2019-09-19 17:30:00,"511 Forest St, San Francisco, CA 94016",1730,CA,San Francisco,19,9
185949,259357,USB-C Charging Cable,1,11.95,2019-09-30 00:18:00,"250 Meadow St, San Francisco, CA 94016",18,CA,San Francisco,30,9


In [13]:
df_5 = df_5.sort_values(by='Mes')
df_5.tail()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia,Mes
38576,303649,USB-C Charging Cable,1,11.95,2019-12-31 13:07:00,"272 Park St, Seattle, WA 98101",1307,WA,Seattle,31,12
38575,303648,Wired Headphones,1,11.99,2019-12-23 09:17:00,"800 Ridge St, San Francisco, CA 94016",917,CA,San Francisco,23,12
38574,303648,Vareebadd Phone,1,400.00,2019-12-23 09:17:00,"800 Ridge St, San Francisco, CA 94016",917,CA,San Francisco,23,12
38596,303668,AA Batteries (4-pack),1,3.84,2019-12-02 07:46:00,"260 Center St, Boston, MA 02215",746,MA,Boston,2,12
51369,315951,AAA Batteries (4-pack),1,2.99,2019-12-20 11:52:00,"598 Wilson St, San Francisco, CA 94016",1152,CA,San Francisco,20,12


In [14]:
df_5['Meses'] = df_5['Fecha de Pedido'].dt.month
df_5['Dias'] = df_5['Fecha de Pedido'].dt.day

# Agrupar por mes y día y calcular la suma de las ventas diarias
ventas_diarias_por_mes = df_5.groupby(['Meses', 'Dias'])['Cantidad Pedida'].sum().reset_index()


In [33]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

In [38]:
import plotly.express as px

fig = px.scatter(ventas_diarias_por_mes, x='Dias', y='Cantidad Pedida', color='Meses', title='Ventas Diarias por Mes')
fig.show()


In [56]:
import plotly.graph_objects as go

# Lista de meses
meses_df = df_5['Meses'].unique()

# Crear un gráfico de barras para cada mes
for mes in meses_df:
    # Filtrar el DataFrame para el mes actual
    df_mes = df_5[df_5['Meses'] == mes]

    # Calcular la suma de las ventas por día para el mes actual
    ventas_diarias = df_mes.groupby('Dias')['Cantidad Pedida'].sum()

    # Crear el gráfico de barras
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=ventas_diarias.index,
        y=ventas_diarias.values,
        name=f'Ventas - Mes {mes}',
                marker=dict(
            color=ventas_diarias.values,  # Usar los valores de ventas como colores
            colorscale= 'RdYlGn',  # Especificar el colorscale
            colorbar=dict(title='Ventas'),  # Agregar barra de colores
        )
    ))

    # Configurar el diseño del gráfico
    fig.update_layout(
        title=f'Ventas Diarias - Mes {mes}',
        xaxis_title='Día',
        yaxis_title='Ventas Diarias',
    )

    # Mostrar el gráfico
    fig.show()


In [67]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Lista de meses
meses_df = df_5['Meses'].unique()

# Crear un gráfico de subplots con 3 filas y 4 columnas
fig = make_subplots(rows=3, cols=4, subplot_titles=meses)

# Iterar a través de los meses y crear un gráfico de barras para cada uno
for i, mes in enumerate(meses_df, 1):
    # Filtrar el DataFrame para el mes actual
    df_mes = df_5[df_5['Meses'] == mes]

    # Calcular la suma de las ventas por día para el mes actual
    ventas_diarias = df_mes.groupby('Dias')['Cantidad Pedida'].sum()

    # Obtener la fila y la columna del subplot actual
    row = (i - 1) // 4 + 1
    col = (i - 1) % 4 + 1

    # Agregar el gráfico de barras al subplot correspondiente
    fig.add_trace(go.Bar(
        x=ventas_diarias.index,
        y=ventas_diarias.values,
        name=f'Ventas - Mes {mes}',
        marker=dict(
            color=ventas_diarias.values,  # Usar los valores de ventas como colores
            colorscale= 'RdYlGn',  # Especificar el colorscale
            colorbar=dict(title='Ventas'),  # Agregar barra de colores

        )
    ), row=row, col=col)

    # Configurar el diseño del gráfico del subplot actual
    fig.update_xaxes( title_text='Dia',row=row, col=col, title_standoff=1)
    fig.update_yaxes(title_text='Ventas', row=row, col=col, range=[280, 1100])

# Configurar el diseño del gráfico general
fig.update_layout(
    title='Ventas Diarias por Mes',
    showlegend=False,  # Ocultar la leyenda individual de cada mes
)

# Mostrar el gráfico
fig.show()



In [68]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Lista de meses
meses_df = df_5['Meses'].unique()

# Crear un gráfico de subplots con 3 filas y 4 columnas
fig = make_subplots(rows=3, cols=4, subplot_titles=meses)

# Iterar a través de los meses y crear un gráfico de barras para cada uno
for i, mes in enumerate(meses_df, 1):
    # Filtrar el DataFrame para el mes actual
    df_mes = df_5[df_5['Meses'] == mes]

    # Calcular la suma de las ventas por día para el mes actual
    ventas_diarias = df_mes.groupby('Dias')['Cantidad Pedida'].sum()

    # Obtener la fila y la columna del subplot actual
    row = (i - 1) // 4 + 1
    col = (i - 1) % 4 + 1

    # Agregar el gráfico de barras al subplot correspondiente
    fig.add_trace(go.Bar(
        x=ventas_diarias.index,
        y=ventas_diarias.values,
        name=f'Ventas - Mes {mes}',
        marker=dict(
            color=ventas_diarias.values,  # Usar los valores de ventas como colores
            colorscale='RdYlGn',  # Especificar el colorscale
            showscale=True,  # Mostrar la barra de colores
            colorbar=dict(
                title='Ventas',  # Título vacío
                tickvals=[ventas_diarias.min(), ventas_diarias.max()],  # Mostrar solo valores mínimo y máximo
                ticktext=['min', 'max'],  # Etiquetas personalizadas
            ),
        )
    ), row=row, col=col)

    # Configurar el diseño del gráfico del subplot actual
    fig.update_xaxes(title_text='Dia', row=row, col=col, title_standoff=1)
    fig.update_yaxes(title_text='Ventas', row=row, col=col, range=[200, 1100])

# Configurar el diseño del gráfico general
fig.update_layout(
    title='Ventas Diarias por Mes',
    showlegend=False,  # Ocultar la leyenda individual de cada mes
)

# Mostrar el gráfico
fig.show()



In [26]:
# Funcion para obtener los datos de dia y mes de los pedidos

def extraer_datos_dia(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Mes'] = df_ventas['Fecha de Pedido'].dt.hour
  datos['Dia'] = df_ventas['Fecha de Pedido'].dt.day
  return datos

In [27]:
# Crear una lista para almacenar los totales de pedidos por día
TotalPedidosDia_ = []

# Iterar a través de los meses
for mes in meses:
    # Obtener los datos para el mes actual
    datos_mes = dataframes[mes]

    # Utilizar la función extraer_datos_dia para agregar las columnas 'Mes' y 'Dia'
    datos_mes = extraer_datos_dia(datos_mes)

    # Calcular el total de pedidos por día
    total_pedidos_dia = datos_mes.groupby('Dia').size().reset_index(name='TotalPedidos')

    # Crear una lista con 31 entradas, una para cada día del mes
    pedidos_por_dia = [0] * 31

    # Llenar la lista pedidos_por_dia con los totales de pedidos correspondientes
    for index, row in total_pedidos_dia.iterrows():
        dia = int(row['Dia'])  # Convertir dia a entero
        total = row['TotalPedidos']
        pedidos_por_dia[dia - 1] = total  # Restar 1 porque los días comienzan desde 1, pero las listas en Python son indexadas desde 0

    # Agregar la lista de totales de pedidos por día a TotalPedidosDia
    TotalPedidosDia_.append(pedidos_por_dia)

# Ahora, TotalPedidosDia contendrá las cantidades de pedidos por día para todos los meses en el mismo orden que la lista de meses

print("Cantidades de Pedidos por Día:", TotalPedidosDia_)


Cantidades de Pedidos por Día: [[315.0, 324.0, 312.0, 316.0, 315.0, 320.0, 302.0, 318.0, 339.0, 327.0, 355.0, 314.0, 321.0, 339.0, 338.0, 310.0, 335.0, 320.0, 333.0, 308.0, 314.0, 305.0, 320.0, 320.0, 293.0, 320.0, 335.0, 317.0, 345.0, 307.0, 0], [392.0, 401.0, 378.0, 383.0, 357.0, 383.0, 374.0, 379.0, 398.0, 389.0, 409.0, 385.0, 392.0, 402.0, 412.0, 373.0, 397.0, 386.0, 403.0, 388.0, 370.0, 360.0, 381.0, 385.0, 354.0, 384.0, 409.0, 370.0, 410.0, 377.0, 0], [390.0, 399.0, 378.0, 382.0, 358.0, 383.0, 372.0, 379.0, 398.0, 389.0, 407.0, 386.0, 393.0, 401.0, 410.0, 371.0, 397.0, 385.0, 405.0, 389.0, 368.0, 361.0, 378.0, 387.0, 353.0, 384.0, 410.0, 370.0, 411.0, 379.0, 0], [389.0, 400.0, 378.0, 385.0, 358.0, 382.0, 373.0, 379.0, 400.0, 392.0, 411.0, 386.0, 392.0, 400.0, 410.0, 369.0, 397.0, 382.0, 404.0, 387.0, 366.0, 356.0, 381.0, 387.0, 351.0, 385.0, 409.0, 370.0, 410.0, 380.0, 0], [390.0, 397.0, 380.0, 384.0, 356.0, 382.0, 373.0, 381.0, 398.0, 391.0, 409.0, 384.0, 390.0, 399.0, 412.0, 37

In [79]:

# Definir los días del mes
dias = list(range(1, 32))

# Crear subplots con 3 filas y 4 columnas
fig = make_subplots(rows=3, cols=4, subplot_titles=meses)

for i, mes in enumerate(meses, 1):
    # Obtener el índice del subplot actual
    row = (i - 1) // 4 + 1
    col = (i - 1) % 4 + 1

    # Obtener los datos del DataFrame para el mes actual
    dataframe_mes = dataframes[mes]

    # Calcular el máximo de ventas para este mes
    dataframe_max = dataframe_mes['Cantidad Pedida'].max()

    # Inicializar una lista para almacenar las ventas diarias
    ventas_diarias = []

    # Calcular las ventas diarias para cada día del mes
    for dia in dias:
        ventas_dia = dataframe_mes[dataframe_mes['Dia'] == dia]['Cantidad Pedida'].sum()
        ventas_diarias.append(ventas_dia)

    # Agregar un gráfico de barras para el mes actual
    fig.add_trace(go.Bar(
        x=dias,
        y=ventas_diarias,
        name=mes,
    ), row=row, col=col)

    # Configurar ejes y títulos para cada subplot
    fig.update_xaxes(title_text="Día", row=row, col=col)
    fig.update_yaxes(title_text="Cantidad", row=row, col=col, range=[250, 500])

# Configurar título general
fig.update_layout(
    title='Productos por día por mes',
    width=1500,  # Ancho en píxeles
    height=800,  # Alto en píxeles
)

fig.show()




In [94]:
import pandas as pd

# Suponiendo que ya has concatenado tus DataFrames en df_total
# Ordenar df_total por la columna 'Fecha de Pedido' en orden ascendente
df_total.sort_values(by='Fecha de Pedido', ascending=True, inplace=True)

# Mostrar las primeras filas del DataFrame ordenado
df_total.columns

NameError: ignored

In [95]:
df_total5 = pd.concat(dataframes.values(), ignore_index=True)
df_total5.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia,Mes
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",846,TX,Dallas,17.0,14.0
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",2230,MA,Boston,16.0,17.0
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",1438,CA,Los Angeles,27.0,7.0
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",1438,CA,Los Angeles,1.0,19.0
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",927,CA,Los Angeles,13.0,14.0


In [101]:
import plotly.graph_objs as go

# Calcular las ventas diarias a lo largo de todos los meses
ventas_diarias_por_dia = df_total5.groupby(['Dia'])['Cantidad Pedida'].sum()

# Crear un gráfico de barras apiladas
data = []
for day in ventas_diarias_por_dia:
    trace = go.Bar(
        x=ventas_diarias_por_dia.index,
        y=ventas_diarias_por_dia[day],
        name=f'Día {day}',
        marker=dict(opacity=0.5)  # Ajustar la opacidad de las barras
    )
    data.append(trace)

layout = go.Layout(
    title='Ventas Diarias por Día a lo largo de los Meses',
    xaxis=dict(title='Mes de Pedido'),
    yaxis=dict(title='Ventas Diarias'),
    barmode='stack'  # Establecer el modo de las barras
)

fig = go.Figure(data=data, layout=layout)

fig.show()


KeyError: ignored

In [103]:
import plotly.graph_objs as go

# Calcular las ventas diarias a lo largo de todos los meses
ventas_diarias_por_dia = df_total5.groupby(['Mes', 'Dia'])['Cantidad Pedida'].sum().unstack()

# Crear un gráfico de barras apiladas
data = []
for column in ventas_diarias_por_dia.columns:
    trace = go.Bar(
        x=ventas_diarias_por_dia.index,
        y=ventas_diarias_por_dia[column],
        name=f'Día {column}',
        marker=dict(opacity=0.5)  # Ajustar la opacidad de las barras
    )
    data.append(trace)

layout = go.Layout(
    title='Ventas Diarias por Día a lo largo de los Meses',
    xaxis=dict(title='Mes de Pedido'),
    yaxis=dict(title='Ventas Diarias'),
    barmode='stack'  # Establecer el modo de las barras
)

fig = go.Figure(data=data, layout=layout)

fig.show()


In [105]:
import plotly.graph_objs as go

# Calcular las ventas diarias a lo largo de todos los meses
ventas_diarias_por_dia = df_total5.groupby(['Mes', 'Dia'])['Cantidad Pedida'].sum().unstack()

# Crear un gráfico de barras para cada día
data = []
for day in ventas_diarias_por_dia.columns:
    trace = go.Bar(
        x=ventas_diarias_por_dia.index,
        y=ventas_diarias_por_dia[day],
        name=f'Día {day}',
        marker=dict(opacity=0.5)  # Ajustar la opacidad de las barras
    )
    data.append(trace)

layout = go.Layout(
    title='Ventas Diarias por Día a lo largo de los Meses',
    xaxis=dict(title='Mes de Pedido'),
    yaxis=dict(title='Ventas Diarias'),
    barmode='group'  # Establecer el modo de las barras
)

fig = go.Figure(data=data, layout=layout)

fig.show()


In [108]:
import plotly.graph_objs as go

# Supongamos que tienes un DataFrame llamado df con datos de ventas para cada día del año

# Calcular las ventas diarias
ventas_diarias = df_total5.groupby(['Dia'])['Cantidad Pedida'].sum()

# Crear un gráfico de barras
fig = go.Figure(data=[go.Bar(
    x=ventas_diarias.index,
    y=ventas_diarias,
)])

# Configurar el diseño del gráfico
fig.update_layout(
    title='Ventas Diarias para Todos los Días del Año',
    xaxis=dict(title='Día del Año'),
    yaxis=dict(title='Ventas Diarias'),
)

# Mostrar el gráfico
fig.show()

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go

# Calcular las ventas diarias a lo largo de todos los meses
ventas_diarias_por_dia = df_total5.groupby(['Mes de Pedido', 'Dia de Pedido'])['Cantidad Pedida'].sum().unstack()

# Crear un gráfico de barras apiladas con barras verticales
fig = sp.make_subplots(rows=1, cols=1)

for day in ventas_diarias_por_dia.columns:
    trace = go.Bar(
        x=ventas_diarias_por_dia.index,
        y=ventas_diarias_por_dia[day],
        name=f'Día {day}',
        opacity=0.5,  # Ajustar la opacidad de las barras
        orientation='v'  # Barras verticales
    )
    fig.add_trace(trace)

fig.update_layout(
    title='Ventas Diarias por Día a lo largo de los Meses',
    xaxis=dict(title='Mes de Pedido'),
    yaxis=dict(title='Ventas Diarias'),
    barmode='stack'  # Establecer el modo de las barras
)

fig.show()


### 6. Impacto de eventos especiales en las ventas:

  ● ¿Se ha observado algún aumento o disminución significativa en las ventas en días
  cercanos a eventos especiales, como días festivos?

  ● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
  las ventas y cómo se manifestó ese impacto?